In [1]:
from sklearn.svm import LinearSVC
from gensim.models import Word2Vec
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
#from abstract import *
from sklearn.datasets import fetch_20newsgroups
from nltk.corpus import stopwords
from sklearn.grid_search import GridSearchCV
import numpy as np
from sklearn.cross_validation import cross_val_score, KFold
from sklearn.metrics import classification_report
import sys



In [2]:
categories = ['alt.atheism','rec.sport.baseball','talk.politics.mideast','comp.graphics', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train',categories=categories, remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test',categories=categories, remove=('headers', 'footers', 'quotes'))

In [3]:
from pprint import pprint
#pprint(newsgroups_train)

# Preprocessing 

In [26]:
import string
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import PorterStemmer


def preprocessor(text, stem=False, stop=False, sent=False):
    """
    Tokenize :text use nltk.word_tokenize, optionally:
        stemming tokens if :stem
        removeing stop words if :stop
        joining tokens into one string with whitespace if :sent
    """
    
    # Remove punctuations
    exclude = set(string.punctuation)
    text = ''.join(ch for ch in text if ch not in exclude)
 
    tokens = word_tokenize(text)
    
    if stop:
        stop = stopwords.words('english')
        tokens =[word for word in tokens if word not in stop]
        tokens = [word.lower() for word in tokens]

    if stem:
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(t) for t in tokens]
    
    if sent:
        tokens = ' '.join(tokens)
        
    return tokens
    



In [27]:
pprint(preprocessor(newsgroups_train.data[0], sent=True, stem=True))
# import nltk
# nltk.download()

u'forgiv ani inaccuraci i delet the origin post isnt thi the same person who wrote the book and wa censur in canada a few year back'



# Using TF-IDF 

In [28]:
from functools import partial
vectorizer = TfidfVectorizer(tokenizer=partial(preprocessor))  # stem=True, stop=True
le = LabelEncoder()

# Training 
vectors_train = vectorizer.fit_transform(newsgroups_train.data)
labels_train = le.fit_transform(newsgroups_train.target)

# Just for debug quickly reassign labels
X = vectors_train
y = labels_train


In [21]:

#clf = MultinomialNB(alpha=.01)
clf = svm.SVC(kernel='linear', C=0.05)

# Evaluate using cross validation
cross_val = KFold(len(newsgroups_train.target), n_folds=3, shuffle=True)

for train_index, test_index in cross_val:
    clf = svm.SVC(kernel='linear', C=0.05)
    X_train, X_test = X[train_index],X[test_index]
    y_train, y_test = y[train_index],y[test_index]
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    print classification_report(le.inverse_transform(y_test),le.inverse_transform(y_pred))
    print metrics.confusion_matrix(y_pred,y_test)
    break


             precision    recall  f1-score   support

          0       1.00      0.02      0.04       145
          1       1.00      0.03      0.05       213
          2       0.21      1.00      0.35       186
          3       1.00      0.02      0.04       205
          4       1.00      0.18      0.31       191

avg / total       0.84      0.25      0.16       940

[[  3   0   0   0   0]
 [  0   6   0   0   0]
 [142 207 186 201 156]
 [  0   0   0   4   0]
 [  0   0   0   0  35]]


In [24]:
clf = svm.SVC(kernel='linear', C=1.)
clf.fit(vectors_train, newsgroups_train.target)
vectors_test = vectorizer.transform(newsgroups_test.data)
pred = clf.predict(vectors_test)

print classification_report(le.inverse_transform(newsgroups_test.target),le.inverse_transform(pred))
# stem and stop 0.82 (.78 - .86)
# no stem or stop .81

             precision    recall  f1-score   support

          0       0.70      0.72      0.71       319
          1       0.87      0.85      0.86       389
          2       0.80      0.87      0.84       397
          3       0.78      0.81      0.79       394
          4       0.88      0.76      0.82       376

avg / total       0.81      0.81      0.81      1875



# Using Doc2Vec

In [29]:
#Preprocessing for Doc2Vec 

newsgroups_train_tokens = map(partial(preprocessor, stop=True), newsgroups_train.data)
newsgroups_test_tokens = map(partial(preprocessor, stop=True), newsgroups_test.data)

In [30]:
from gensim.models import Doc2Vec
import gensim.models.doc2vec

from collections import OrderedDict
import multiprocessing
from gensim.models.doc2vec import TaggedDocument
import sys

cores = multiprocessing.cpu_count()
assert gensim.models.doc2vec.FAST_VERSION > -1, "this will be painfully slow otherwise"


In [33]:
from collections import namedtuple
import nltk
from gensim.models.doc2vec import LabeledSentence


In [32]:
?LabeledSentence

In [35]:
from collections import namedtuple
import nltk
from gensim.models.doc2vec import LabeledSentence


def convert_newsgroup(docs, split):
    """
    Generate doc2vec.LabeledSentence instances from dataset :docs
    with train/test indicator :split
    """
    for i,v in enumerate(docs):
        label = '%s_%s' % (split, i)
        yield LabeledSentence(v, [label])

test_docs = list(convert_newsgroup(newsgroups_test_tokens, 'test'))
train_docs = list(convert_newsgroup(newsgroups_train_tokens, 'train'))

all_newsgroup_documents = [train_docs, test_docs]

doc_list = all_newsgroup_documents[:]  # for reshuffling per pass

print('%d doc sets: %d train, %d test' % (len(doc_list), len(train_docs), len(test_docs)))


2 doc sets: 2818 train, 1875 test


### Set Up Doc2Vec

In [86]:
#Doc2Vec(dm=0, dm_concat=1, size=300, window=5, negative=5, hs=0, min_count=3, workers=cores)
#Doc2Vec(dm=0, dm_mean=1, size=300, window=5, negative=5, hs=0, min_count=3, workers=cores),

dbow_model = Doc2Vec(dm=0, dm_concat=0, sample=1e-5, size=288, window=5, negative=5, hs=0, min_count=2, workers=cores)
# dm_model =  Doc2Vec(dm=1, dm_mean=1, sample=1e-5, size=300, window=10, negative=5, hs=0, min_count=2, workers=cores)

# TODO speed setup by sharing results of 1st model's vocabulary scan
# dbow_model.load_word2vec_format('/home/skillachie/Downloads/GoogleNews-vectors-negative300.bin.gz', binary=True)
dbow_model.build_vocab(train_docs)  # PV-DM/concat requires one special NULL word so it serves as template

# dm_model.load_word2vec_format('/home/skillachie/Downloads/GoogleNews-vectors-negative300.bin.gz', binary=True)
# dm_model.build_vocab(all_newsgroup_documents)  # PV-DM/concat requires one special NULL word so it serves as template



# Models to evaluate
#simple_models = [
 
    # PV-DBOW  0.86 with Stem & hs=0
    #Doc2Vec(dm=0, dm_concat=1,sample=1e-5, size=300, window=5, negative=5, hs=0, min_count=2, workers=cores),
    
    #
    #Doc2Vec(dm=0, dm_mean=1, sample=1e-5,size=300, window=5, negative=5, hs=0, min_count=2, workers=cores),
    
    
    #Doc2Vec(dm=0, size=300, negative=5, hs=0, min_count=1, workers=cores),
    
    # PV-DM w/average No good 0.84
    #Doc2Vec(dm=1, dm_mean=1, sample=1e-5, size=300, window=10, negative=5, hs=0, min_count=2, workers=cores),
    
    # PV-DM w/sum
    #Doc2Vec(dm=1, dm_mean=0, size=100, window=10, negative=5, hs=0, min_count=2, workers=cores),
#]

# speed setup by sharing results of 1st model's vocabulary scan
#simple_models[0].load_word2vec_format('/home/skillachie/Downloads/GoogleNews-vectors-negative300.bin.gz', binary=True)
#simple_models[0].build_vocab(all_newsgroup_documents)  # PV-DM/concat requires one special NULL word so it serves as template


#print(simple_models[0])

#for model in simple_models[1:]:
    #model.reset_from(simple_models[0])
#    model.load_word2vec_forma('/home/skillachie/nlpArea51/doc2vec/GoogleNews-vectors-negative300.bin', binary=True)
#    model.build_vocab(all_newsgroup_documents)
#    print(model)

#models_by_name = OrderedDict((str(model), model) for model in simple_models)

In [10]:
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
dbow_dmm_model = ConcatenatedDoc2Vec([dbow_model, dm_model])
#models_by_name['dbow+dmc'] = ConcatenatedDoc2Vec([simple_models[1], simple_models[0]])

### Prediction and Evaluation

### Training

Why is bulk training done with all the  docs from train, test,  and dev ? Compared to other methods training is only done on the training dataset 

In [38]:
from collections import defaultdict
best_error = defaultdict(lambda: 1.)  # to selectively-print only best errors achieved

In [47]:

def extract_vectors(model, docs):
    """
    Get Vectors From Word2Vec
    """
    vectors_list = []
    
    for doc_no in range(len(docs)):
        doc_label = docs[doc_no].tags[0]
        doc_vector = model.docvecs[doc_label]
        vectors_list.append(doc_vector)
        
    return vectors_list

# TODO inferred vectors

def get_infer_vectors(model, docs):
    
    vecs = []
    for doc in docs:
        vecs.append(model.infer_vector(doc.words))
    return vecs
    
    

In [44]:
# dbow_model.alpha, dbow_model.min_alpha = 0.025, 0.025
# dbow_model.train(train_docs)
?dbow_model.train

In [89]:
from random import shuffle
alpha_delta = (alpha - min_alpha) / passes
# train_docs_ = list(train_docs)
for gen in range(10):
    alpha, min_alpha, passes = (0.025, 0.001, 50)
    for epoch in range(passes):
        shuffle(train_docs_)

        #for name, train_model in models_by_name.items():

        #Train
        print alpha
        dbow_model.alpha, dbow_model.min_alpha = alpha, alpha
        print(dbow_model.train(train_docs_))

    #     dm_model.alpha, dm_model.min_alpha = alpha, alpha
    #     dm_model.train(doc_list)


    #     dbow_dmm_model.alpha, dbow_dmm_model.min_alpha = alpha, alpha
    #     dbow_dmm_model.train(doc_list)


        alpha -= alpha_delta


0.025
638185
0.0254512
636462
0.0259024
638580
0.0263536
637404
0.0268048
637253
0.027256
637970
0.0277072
638143
0.0281584
637638
0.0286096
637298
0.0290608
637119
0.029512
637505
0.0299632
637583
0.0304144
636485
0.0308656
637135
0.0313168
637124
0.031768
636422
0.0322192
637863
0.0326704
636637
0.0331216
637577
0.0335728
637041
0.034024
637265
0.0344752
637536
0.0349264
637325
0.0353776
637330
0.0358288
636912
0.03628
637715
0.0367312
637049
0.0371824
637141
0.0376336
638616
0.0380848
637745
0.038536
637273
0.0389872
636911
0.0394384
636937
0.0398896
637741
0.0403408
637115
0.040792
637771
0.0412432
637026
0.0416944
637000
0.0421456
637824
0.0425968
637803
0.043048
637627
0.0434992
637283
0.0439504
638016
0.0444016
637662
0.0448528
637804
0.045304
637704
0.0457552
637236
0.0462064
637016
0.0466576
637395
0.0471088
636903
0.025
637537
0.0254512
637813
0.0259024
637340
0.0263536
637258
0.0268048
637152
0.027256
637140
0.0277072
636351
0.0281584
637591
0.0286096
637227
0.0290608
636811

KeyboardInterrupt: 

In [83]:
'hitter' in dbow_model.vocab

True

In [82]:
for d in newsgroups_train.data:
    if 'baseball' in d:
        print d
        break



I'm not quite sure how these numbers are generated.  It appears that in
a neutral park Bo's HR and slugging tend to drop (he actually loses two
home runs).  Or do they?  What is "equivalent average?"

One thing, when looking at Bo's stats, is that you can see that KC took
away some homers.  Normally, you expect some would-be homers to go for
doubles or triples in big parks, or to be caught, and for that matter you
expect lots of doubles and triples anyway.  But Bo, despite his speed, 
hit very few doubles and not that many triples.  So I would expect his
value to have risen quite considerably in a neutral park.  


Felix Jose has been a .350/.440 player in a fairly neutral park.
I would offhand guess the `89-`90 Bo at around a .330/.530 player.
Maybe .330/.550 .  Not even close.


I'd put him about there too.  

Note: I hadn't realized the media had hyped him so much.  I thought he
was always viewed by them as a better football player, and only so-so 
at baseball.  He did only have o

In [90]:
"""
god belief graphic reason
graphic
graphic
pitch hitter clutch heretic
clutch
clutch
congress hardware vote conservative
congress
congress
shuttle launch fishing solar
fishing
fishing
"""

# categories = ['alt.atheism','rec.sport.baseball','talk.politics.mideast','comp.graphics', 'sci.space']
untrained_model = Doc2Vec(dm=0, dm_concat=0, sample=1e-5, size=48, window=5, negative=5, hs=0, min_count=2, workers=cores)
untrained_model.build_vocab(train_docs)
for sample in [
    'god belief graphic reason',
    'pitch hitter clutch heretic',
    'congress hardware vote conservative',
    'shuttle launch fishing solar'
]:
    print(sample)
    print(untrained_model.doesnt_match(sample.split()))
    print(dbow_model.doesnt_match(sample.split()))    
# dbow_model.doesnt_match('god belief graphic reason'.split())
# newsgroups_train.filenames



god belief graphic reason
graphic
graphic
pitch hitter clutch heretic
clutch
clutch
congress hardware vote conservative
congress
congress
shuttle launch fishing solar
fishing
fishing


In [93]:
#Evaluation
train_vectors = extract_vectors(untrained_model, train_docs)
test_vectors = get_infer_vectors(untrained_model, test_docs)


#model = LinearSVC()
#penalties = np.array([0.001,0.002,0.003,0.004,0.005,0.007,0.008,0.009,0.01,0.05,0.04,0.03,0.02])
#grid = GridSearchCV(estimator=model ,n_jobs=7,param_grid=dict(C=penalties))
#grid.fit(train_vectors, newsgroups_train.target)
        
# summarize the results of the grid search
#print(grid.best_score_)
#print(grid.best_estimator_.C)

#clf = LinearSVC(C=0.009)
clf = LinearSVC(C=0.0025)
clf.fit(train_vectors, newsgroups_train.target)

predDoc = clf.predict(test_vectors)
        
print classification_report(le.inverse_transform(newsgroups_test.target), le.inverse_transform(predDoc))

# avg / total       0.77      0.76      0.76      1875
# avg / total       0.78      0.76      0.76      1875

             precision    recall  f1-score   support

          0       0.00      0.00      0.00       319
          1       0.00      0.00      0.00       389
          2       0.21      1.00      0.35       397
          3       0.00      0.00      0.00       394
          4       0.00      0.00      0.00       376

avg / total       0.04      0.21      0.07      1875



In [92]:
print classification_report(le.inverse_transform(newsgroups_train.target), le.inverse_transform(clf.predict(train_vectors)))

             precision    recall  f1-score   support

          0       0.87      0.85      0.86       480
          1       0.93      0.92      0.93       584
          2       0.92      0.91      0.91       597
          3       0.94      0.87      0.90       593
          4       0.81      0.91      0.85       564

avg / total       0.90      0.89      0.89      2818

